# Pneumonia Model - Results Analysis with FCP
Demonstrate usage of the Rhino Health Python SDK for analyzing code runs using federated analytics

#### Prerequisites 
1. Run training and validation of a pneumonia detection model (e.g. from Tutorial 1)
2. Have an output dataset from the model validation with 'Pneumonia' and 'ModelScore' fields
3. Have a code run to which you can upload the model performance analysis report

### Initialization and Login

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.axes
import matplotlib.figure
import matplotlib.pyplot as plt
from PIL import Image
import os
import sys
import base64
import json
from getpass import getpass
import rhino_health
from rhino_health.lib.metrics import RocAuc, RocAucWithCI

In [ ]:
print("Logging In")
my_username = "my_email@example.com" # Replace this with the email you use to log into Rhino Health
session = rhino_health.login(username=my_username, password=getpass.getpass())
print("Logged In")

### Load the results dataset from the project 

In [ ]:
site1_results_dataset = 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX' # Replace dataset UID
dataset = session.dataset.get_dataset(site1_results_dataset)
print(f"Loaded dataset '{dataset.name}'")

### Calculate ROC (underlying results data stays on-prem)

In [ ]:
metric_configuration = RocAuc(y_true_variable="Pneumonia", y_pred_variable="ModelScore")
results = dataset.get_metric(metric_configuration)

### Plot the ROC with your favorite plotting tool

In [ ]:
roc_metrics = results.output

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
color = colors[0]
linestyle = linestyle_cycle[0]
ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
ax.title.set_text('Overall ROC')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

fig.canvas.draw()
image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
image_to_store.save("Overall_ROC.png", format='png', optimize=True, quality=100)

### Calculate and plot ROC with a Confidence Interval

In [ ]:
metric_configuration = RocAucWithCI(
   timeout_seconds = 30.0,
    y_true_variable="Pneumonia",
    y_pred_variable="ModelScore",
    confidence_interval=95
)
results = dataset.get_metric(metric_configuration)

In [ ]:
roc_metrics = results.output

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
color = colors[0]
linestyle = linestyle_cycle[0]
tpr_ci = roc_metrics['tpr_ci']
ax.fill_between(roc_metrics['fpr'], tpr_ci[0], tpr_ci[1], alpha=0.33, label='_nolegend_', color=color)
ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
ax.title.set_text('Overall ROC with Confidence Interval')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
fig.canvas.draw()
image_to_store = Image.frombytes('RGB', 
fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
image_to_store.save("Overall_ROC_CI.png", format='png', optimize=True, quality=100)

### Calculate and plot ROC grouped by Gender

In [ ]:
metric_configuration = RocAuc(y_true_variable="Pneumonia",
                              y_pred_variable="ModelScore",
                             group_by = { 'groupings': ['Gender']})
results = dataset.get_metric(metric_configuration)

In [ ]:
for group in results.output.keys():
    roc_metrics= results.output[group]
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    linestyle_cycle = ['-', '--']
    fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
    color = colors[0]
    linestyle = linestyle_cycle[0]
    ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle)
    ax.title.set_text(group)
    ax.set_xlabel('1 - Specificity')
    ax.set_ylabel('Sensitivity')
    ax.grid(True)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    fig.canvas.draw()
    image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    image_to_store.save(f"Gender_ROC_{group}.png", format='png', optimize=True, quality=100)


### Plot the ROC by Gender on a single chart

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
linestyle_cycle = ['-', '--']
fig, ax =plt.subplots(figsize=[6, 4], dpi=200)

linestyle = linestyle_cycle[0]
for group,color in zip(results.output.keys(),colors):
    roc_metrics= results.output[group]
    ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=color, linestyle=linestyle,label = group)
    ax.legend(loc='lower right')
ax.title.set_text('ROC by Gender')
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')
ax.grid(True)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

fig.canvas.draw()
image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
image_to_store.save("Gender_ROC.png", format='png', optimize=True, quality=100)


### Upload the visualizations to FCP

In [ ]:
print("Sending visualizations to the Cloud")
code_result_uid = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"  # Replace code runs UID

def add_images_to_report(report_data, image_files):
    for image_file in image_files:
        with open(image_file, "rb") as temp_image:
            base_64_image = base64.b64encode(temp_image.read()).decode("utf-8")
            report_data.append(
              {
                 "type": "Image",
                 "data": {
                     "image_filename": image_file,
                     "image_base64": base_64_image,
                 },
                 "width": 100 / len(image_files)
              }
           )

roc_image_files = ('Overall_ROC.png', 'Overall_ROC_CI.png')
gender_image_files = ('Gender_ROC_M.png', 'Gender_ROC_F.png', 'Gender_ROC.png')

report_data = []
report_data.append({"type": "Title", "data": "Overall ROC"})
add_images_to_report(report_data, roc_image_files)
report_data.append({"type": "Title", "data": "ROC by Gender"})
add_images_to_report(report_data, gender_image_files)
    

result = session.post(f"code_runs/{code_result_uid}/set_report/", 
                      data={"report_data": json.dumps(report_data)})
print('Done')